In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
dataset_path = kagglehub.dataset_download('masoudnickparvar/brain-tumor-mri-dataset')

print('Data source import complete.')
import shutil

destination = "/content/brain-tumor-mri-dataset"
shutil.move(dataset_path, destination)
print(f"Dataset moved to {destination}")


100%|██████████| 149M/149M [00:01<00:00, 153MB/s]

Extracting files...


Data source import complete.
Dataset moved to /content/brain-tumor-mri-dataset


# Import Needed Libraries

In [2]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import warnings
warnings.filterwarnings("ignore")

# Data Preprocessing

### Read data and store it in dataframe

In [3]:
train_data_dic = '/content/brain-tumor-mri-dataset/Training'
imagePaths = []
labels =[]

floders = os.listdir(train_data_dic)

for floder in floders:
    floderPath = os.path.join(train_data_dic, floder)
    imageList = os.listdir(floderPath)

    for image in imageList:
        imagePath = os.path.join(floderPath, image)

        imagePaths.append(imagePath)
        labels.append(floder)


# Concatenate image paths with labels into one dataframe
Iseries = pd.Series(imagePaths, name= 'imagepaths')
Lseries = pd.Series(labels, name='labels')

train_df = pd.concat([Iseries, Lseries], axis= 1)

In [4]:
train_df

,imagepaths,labels
0,/content/brain-tumor-mri-dataset/Training/notu...,notumor
1,/content/brain-tumor-mri-dataset/Training/notu...,notumor
2,/content/brain-tumor-mri-dataset/Training/notu...,notumor
3,/content/brain-tumor-mri-dataset/Training/notu...,notumor
4,/content/brain-tumor-mri-dataset/Training/notu...,notumor
...,...,...
5707,/content/brain-tumor-mri-dataset/Training/meni...,meningioma
5708,/content/brain-tumor-mri-dataset/Training/meni...,meningioma
5709,/content/brain-tumor-mri-dataset/Training/meni...,meningioma
5710,/content/brain-tumor-mri-dataset/Training/meni...,meningioma


In [5]:
test_data_dic = '/content/brain-tumor-mri-dataset/Testing'
imagePaths = []
labels =[]

floders = os.listdir(test_data_dic)

for floder in floders:
    floderPath = os.path.join(test_data_dic, floder)
    imageList = os.listdir(floderPath)

    for image in imageList:
        imagePath = os.path.join(floderPath, image)

        imagePaths.append(imagePath)
        labels.append(floder)


# Concatenate image paths with labels into one dataframe
Iseries = pd.Series(imagePaths, name= 'imagepaths')
Lseries = pd.Series(labels, name='labels')

ts_df = pd.concat([Iseries, Lseries], axis= 1)

In [6]:
ts_df

,imagepaths,labels
0,/content/brain-tumor-mri-dataset/Testing/notum...,notumor
1,/content/brain-tumor-mri-dataset/Testing/notum...,notumor
2,/content/brain-tumor-mri-dataset/Testing/notum...,notumor
3,/content/brain-tumor-mri-dataset/Testing/notum...,notumor
4,/content/brain-tumor-mri-dataset/Testing/notum...,notumor
...,...,...
1306,/content/brain-tumor-mri-dataset/Testing/menin...,meningioma
1307,/content/brain-tumor-mri-dataset/Testing/menin...,meningioma
1308,/content/brain-tumor-mri-dataset/Testing/menin...,meningioma
1309,/content/brain-tumor-mri-dataset/Testing/menin...,meningioma


### Split dataframe into train, valid, and test

In [7]:
test_df , valid_df = train_test_split(ts_df, test_size=0.5, shuffle=True, random_state=123)

### Create image data generator

In [8]:
img_size = (224, 224)
batch_size = 16

tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

train_gen = tr_gen.flow_from_dataframe(train_df , x_col="imagepaths" , y_col="labels", target_size=img_size , batch_size=batch_size ,
                                       class_mode="categorical", color_mode= 'rgb' , shuffle= True)

valid_gen = ts_gen.flow_from_dataframe(valid_df , x_col="imagepaths" , y_col="labels", target_size=img_size , batch_size=batch_size ,
                                       class_mode="categorical", color_mode= 'rgb' , shuffle= True)

test_gen = ts_gen.flow_from_dataframe(test_df , x_col="imagepaths" , y_col="labels", target_size=img_size , batch_size=batch_size ,
                                       class_mode="categorical", color_mode= 'rgb' , shuffle= True)

Found 5712 validated image filenames belonging to 4 classes.
Found 656 validated image filenames belonging to 4 classes.
Found 655 validated image filenames belonging to 4 classes.


# Model Structure

In [9]:
img_shape = (224, 224, 3)
class_count = 4

base_model = tf.keras.applications.EfficientNetB4(
            include_top=False,
            weights="imagenet",
            input_shape=img_shape,
            pooling='max'
            )

model = Sequential([
    base_model,
    Dense(256, activation= 'relu'),
    Dropout(rate= 0.45, seed= 123),
    Dense(class_count, activation= 'softmax')
])

model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb4 (Functional)          │ (None, 1792)                │      17,673,823 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         459,008 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │           1,028 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,133,859 (69.18 MB)

 Trainable params: 18,008,652 (68.70 MB)

 Non-trainable params: 125,207 (489.09 KB)

In [10]:
epochs = 10

history = model.fit(train_gen, epochs= epochs, verbose= 1, validation_data= valid_gen, shuffle= False)

Epoch 1/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 225s 217ms/step - accuracy: 0.7579 - loss: 1.5263 - val_accuracy: 0.9573 - val_loss: 0.1120
Epoch 2/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 70s 195ms/step - accuracy: 0.9493 - loss: 0.1458 - val_accuracy: 0.9710 - val_loss: 0.0728
Epoch 3/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 71s 198ms/step - accuracy: 0.9739 - loss: 0.0846 - val_accuracy: 0.9832 - val_loss: 0.0390
Epoch 4/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 71s 198ms/step - accuracy: 0.9858 - loss: 0.0458 - val_accuracy: 0.9863 - val_loss: 0.0418
Epoch 5/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 71s 198ms/step - accuracy: 0.9889 - loss: 0.0372 - val_accuracy: 0.9787 - val_loss: 0.0630
Epoch 6/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 71s 198ms/step - accuracy: 0.9912 - loss: 0.0310 - val_accuracy: 0.9848 - val_loss: 0.0577
Epoch 7/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 71s 199ms/step - accuracy: 0.9935 - loss: 0.0208 - val_accuracy: 0.9909 - val_loss: 0.0255
Epoch 8/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 82s 199ms/step - accuracy: 0.9932 - loss: 

# Evaluate model

In [11]:
train_score = model.evaluate(train_gen, verbose= 1)
valid_score = model.evaluate(valid_gen, verbose= 1)
test_score = model.evaluate(test_gen, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

357/357 ━━━━━━━━━━━━━━━━━━━━ 19s 54ms/step - accuracy: 0.9989 - loss: 0.0034
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.9838 - loss: 0.0770
41/41 ━━━━━━━━━━━━━━━━━━━━ 10s 249ms/step - accuracy: 0.9932 - loss: 0.0180
Train Loss:  0.004623824264854193
Train Accuracy:  0.998774528503418
--------------------
Validation Loss:  0.05041021108627319
Validation Accuracy:  0.9878048896789551
--------------------
Test Loss:  0.03903749957680702
Test Accuracy:  0.9923664331436157


In [13]:
import pickle

with open('/content/model.pkl', 'wb') as f:
  pickle.dump(model, f)


In [15]:
# prompt: model save

# ... (Your existing code) ...

# Save the model to Google Drive
model.save('/content/brain_tumor_model.h5')

# ... (Rest of your code) ...
